# Boilerplate - inicialização padrão

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName("f1-challenge-pt3")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

#### Adicionando arquivo no HDFS

In [3]:
! hadoop fs -put ../datasets/f1

In [4]:
! hadoop fs -ls hdfs://node-master:9000/user/root

Found 2 items
drwxr-xr-x   - root supergroup          0 2023-06-04 16:44 hdfs://node-master:9000/user/root/.sparkStaging
drwxr-xr-x   - root supergroup          0 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1


In [5]:
! hadoop fs -ls hdfs://node-master:9000/user/root/f1

Found 13 items
-rw-r--r--   2 root supergroup       8667 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/circuits.csv
-rw-r--r--   2 root supergroup     224140 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/constructorResults.csv
-rw-r--r--   2 root supergroup     267256 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/constructorStandings.csv
-rw-r--r--   2 root supergroup      15617 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/constructors.csv
-rw-r--r--   2 root supergroup     768136 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/driverStandings.csv
-rw-r--r--   2 root supergroup      79533 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/drivers.csv
-rw-r--r--   2 root supergroup   12118621 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/lapTimes.csv
-rw-r--r--   2 root supergroup     220898 2023-06-04 16:45 hdfs://node-master:9000/user/root/f1/pitStops.csv
-rw-r--r--   2 root supergroup     315477 2023-06-04 16:45 hdfs://node-master:900

#### Verificando a adição no HDFS

In [6]:
files = ["circuits.csv",
"constructorResults.csv",
"constructorStandings.csv",
"constructors.csv",
"driverStandings.csv",
"drivers.csv",
"lapTimes.csv",
"pitStops.csv",
"qualifying.csv",
"races.csv",
"results.csv",
"seasons.csv",
"status.csv",]

In [7]:
! hdfs fsck hdfs://node-master:9000/user/root/f1 -files -blocks

Connecting to namenode via http://node-master:50070/fsck?ugi=root&files=1&blocks=1&path=%2Fuser%2Froot%2Ff1
FSCK started by root (auth:SIMPLE) from /172.18.0.7 for path /user/root/f1 at Sun Jun 04 16:46:29 GMT 2023
/user/root/f1 <dir>
/user/root/f1/circuits.csv 8667 bytes, 1 block(s):  OK
0. BP-896234228-172.18.0.7-1685896813972:blk_1073741830_1006 len=8667 repl=2

/user/root/f1/constructorResults.csv 224140 bytes, 1 block(s):  OK
0. BP-896234228-172.18.0.7-1685896813972:blk_1073741831_1007 len=224140 repl=2

/user/root/f1/constructorStandings.csv 267256 bytes, 1 block(s):  OK
0. BP-896234228-172.18.0.7-1685896813972:blk_1073741832_1008 len=267256 repl=2

/user/root/f1/constructors.csv 15617 bytes, 1 block(s):  OK
0. BP-896234228-172.18.0.7-1685896813972:blk_1073741833_1009 len=15617 repl=2

/user/root/f1/driverStandings.csv 768136 bytes, 1 block(s):  OK
0. BP-896234228-172.18.0.7-1685896813972:blk_1073741834_1010 len=768136 repl=2

/user/root/f1/drivers.csv 79533 bytes, 1 block(s):  O

#### Carregando DataFrame a partir de um caminho no HDFS

In [8]:
HDFS_PATH_F1 = "hdfs://node-master:9000/user/root/f1"

#### Carregando csv's

In [9]:
dfe = {}
for file in files:
    dfe[file[:-4]] = spark.read\
              .options(delimiter=",", header=True, encoding="iso-8859-1")\
              .csv(f"/user/root/f1/{file}")

In [10]:
keys = dfe.keys()

#### Nomes Tabelas

In [11]:
keys

dict_keys(['circuits', 'constructorResults', 'constructorStandings', 'constructors', 'driverStandings', 'drivers', 'lapTimes', 'pitStops', 'qualifying', 'races', 'results', 'seasons', 'status'])

#### Registrando DataFrame como Tabela

In [12]:
for key in keys:
    dfe[key].registerTempTable(key)

#### Pontuação média por corrida dos 20 melhores pilotos das últimas dez temporadas?

In [13]:
spark.sql("""
    WITH last_10_seasons AS (
        SELECT year
        FROM seasons
        ORDER BY year DESC
        LIMIT 10
    ),
    races_last_10_seasons AS (
        SELECT r.*
        FROM races r
        WHERE r.year IN (SELECT year FROM last_10_seasons)
    )
    SELECT r.driverId, d.driverRef, AVG(r.points) AS avg_points, SUM(r.points) AS total_points
    FROM results r
    INNER JOIN races_last_10_seasons l ON r.raceId = l.raceId
    INNER JOIN drivers d ON r.driverId = d.driverId
    GROUP BY r.driverId, d.driverRef
    ORDER BY avg_points DESC
    LIMIT 20
""").show()

+--------+------------------+------------------+------------+
|driverId|         driverRef|        avg_points|total_points|
+--------+------------------+------------------+------------+
|       1|          hamilton|13.890173410404625|      2403.0|
|      20|            vettel|13.780346820809248|      2384.0|
|       3|           rosberg| 10.15359477124183|      1553.5|
|      17|            webber|10.079787234042554|       947.5|
|       8|         raikkonen| 7.774436090225564|      1034.0|
|       4|            alonso| 7.635294117647059|      1298.0|
|     822|            bottas|7.3061224489795915|       716.0|
|     830|    max_verstappen| 7.016666666666667|       421.0|
|      18|            button| 6.512987012987013|      1003.0|
|     817|         ricciardo| 6.325581395348837|       816.0|
|      13|             massa| 5.266666666666667|       869.0|
|       9|            kubica|              4.25|       153.0|
|     815|             perez|3.4338235294117645|       467.0|
|      3

#### Todas as corridas as quais apenas três equipes pontuaram?

In [14]:
spark.sql("""
    SELECT r.raceId, r.year, r.name
    FROM races r
    JOIN (
        SELECT raceId
        FROM results
        GROUP BY raceId
        HAVING COUNT(DISTINCT constructorId) = 3
    ) subquery ON r.raceId = subquery.raceId
""").show()

+------+----+--------------------+
|raceId|year|                name|
+------+----+--------------------+
|   801|1954|  Belgian Grand Prix|
|   790|1956|   German Grand Prix|
|   799|1954|Argentine Grand Prix|
|   827|1951|  Belgian Grand Prix|
|   783|1957|  Italian Grand Prix|
|   765|1958|Argentine Grand Prix|
+------+----+--------------------+



#### Melhor tempo de pitstop registrado, equipe que executou e piloto que estava no carro por temporada?

In [15]:
spark.sql("""
    SELECT pitStops.duration, constructors.name AS team, drivers.driverRef
    FROM pitStops
    JOIN (
        SELECT raceId, MIN(duration) AS best_time
        FROM pitStops
        GROUP BY raceId
    ) AS best_pitstop_times
    ON pitStops.raceId = best_pitstop_times.raceId
    AND pitStops.duration = best_pitstop_times.best_time
    JOIN results ON pitStops.raceId = results.raceId
        AND pitStops.driverId = results.driverId
    JOIN races ON pitStops.raceId = races.raceId
    JOIN constructors ON results.constructorId = constructors.constructorId
    JOIN drivers ON pitStops.driverId = drivers.driverId
    ORDER BY pitStops.duration
    LIMIT 1
""").show()

+--------+--------+---------+
|duration|    team|driverRef|
+--------+--------+---------+
|  12.897|Williams|maldonado|
+--------+--------+---------+



#### Top-5 pilotos que mais vezes fizeram a pole position (primeiro lugar no qualifying e que ganharam a corrida)?

In [16]:
spark.sql("""
    SELECT d.driverRef, COUNT(*) AS total_poles_and_wins
    FROM qualifying q
    JOIN races r ON q.raceId = r.raceId
    JOIN drivers d ON q.driverId = d.driverId
    JOIN results res ON res.raceId = q.raceId AND res.positionOrder = 1 AND res.driverId = q.driverId
    WHERE q.position = 1
    GROUP BY q.driverId, d.driverRef
    ORDER BY total_poles_and_wins DESC
    LIMIT 5
    """).show()

+------------------+--------------------+
|         driverRef|total_poles_and_wins|
+------------------+--------------------+
|          hamilton|                  39|
|            vettel|                  29|
|michael_schumacher|                  24|
|           rosberg|                  15|
|            alonso|                  14|
+------------------+--------------------+



#### Top-5 pilotos que mais vezes se classificaram nas três primeiras posições do qualifying de forma consecutiva (sem intervalo de corridas) das últimas dez temporadas?

In [17]:
spark.sql("""
    WITH qualifying_ranks AS (
        SELECT q.driverId, q.raceId, q.position,
            ROW_NUMBER() OVER (PARTITION BY q.driverId ORDER BY q.raceId) AS rank
        FROM qualifying q
        JOIN races r ON q.raceId = r.raceId
        JOIN seasons s ON r.year = s.year
        WHERE s.year >= YEAR(CURRENT_DATE()) - 10
    ),
    consecutive_qualifying AS (
        SELECT qr.driverId, qr.raceId
        FROM qualifying_ranks qr
        JOIN qualifying_ranks qr_prev
            ON qr.driverId = qr_prev.driverId
            AND qr.rank = qr_prev.rank + 1
            AND qr.raceId = qr_prev.raceId + 1
            AND qr.position <= 3
            AND qr_prev.position <= 3
    )
    SELECT cq.driverId, d.driverRef, COUNT(*) AS total_consecutive_qualifying
    FROM consecutive_qualifying cq
    JOIN drivers d ON cq.driverId = d.driverId
    GROUP BY cq.driverId, d.driverRef
    ORDER BY total_consecutive_qualifying DESC
    LIMIT 5
""").show()

+--------+---------+----------------------------+
|driverId|driverRef|total_consecutive_qualifying|
+--------+---------+----------------------------+
|       1| hamilton|                          58|
|       3|  rosberg|                          54|
|      20|   vettel|                          36|
|     822|   bottas|                          15|
|     817|ricciardo|                           6|
+--------+---------+----------------------------+



#### Top-5 pilotos que mais pontuaram na F1, daqueles que nunca subiram no pódio?

In [18]:
spark.sql("""
    WITH top_drivers AS (
      SELECT driverId, SUM(points) AS total_points
      FROM results
      WHERE driverId NOT IN (
        SELECT DISTINCT driverId
        FROM results
        WHERE position <= 3
      )
      GROUP BY driverId
    )
    SELECT td.driverId, d.driverRef, td.total_points
    FROM top_drivers td
    JOIN drivers d ON td.driverId = d.driverId
    ORDER BY td.total_points DESC
    LIMIT 5
""").show()

+--------+----------+------------+
|driverId| driverRef|total_points|
+--------+----------+------------+
|     807|hulkenberg|       405.0|
|      16|     sutil|       124.0|
|     814|     resta|       121.0|
|     832|     sainz|       118.0|
|     839|      ocon|        87.0|
+--------+----------+------------+

